<a href="https://colab.research.google.com/github/HaydeePeruyero/CAMDA2023/blob/main/resultados/Camda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Leer datos

In [ ]:
#!pip install contextily
#!pip install geopandas

In [ ]:
#from google.colab import drive
#import os
#drive.mount('/content/drive/')

In [ ]:
#os.chdir('/content/drive/MyDrive/CAMDA')
#!pwd

In [1]:
import contextily as ctx
import geopandas as gpd
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
carpeta_data = "data"
nombre_archivo = "amr_counts_complete_20230606.tsv"

# Obtén la ruta absoluta de la carpeta actual
ruta_actual = os.getcwd()

# Construye la ruta completa al archivo CSV dentro de la carpeta "data"
ruta_archivo = os.path.join(ruta_actual, carpeta_data, nombre_archivo)

temp_data = pd.read_table(ruta_archivo, delimiter="\t",index_col=0)
temp_data.shape

#temp_data.head()

(331, 633)

In [5]:
#names_to_drop = ["name","ARO","ARO_Trace","Aro_name_trace","family", "drugClass","mechanism"]
#genes_data = pd.DataFrame(temp_data.drop(names_to_drop, axis=1).transpose())
#genes_data = pd.DataFrame(temp_data)
#genes_data.columns = temp_data['ARO']
#genes_data.index

In [3]:
temp_data.isnull().values.any()

False

In [4]:
from sklearn.preprocessing import LabelEncoder

carpeta_data = "data"
nombre_archivo = "relative_order_assembly.csv"
ruta_actual = os.getcwd()
ruta_archivo = os.path.join(ruta_actual, carpeta_data, nombre_archivo)
otus = pd.read_csv(ruta_archivo, index_col="ID")
temp_y = otus["ID_city"]
# junto las tablas de genes y otus, solo para estar seguro de que tienen las mismas observaciones
genes_data = temp_data.join(temp_y, how="inner")
otus_y = genes_data['ID_city']
genes_data = pd.DataFrame(genes_data.drop('ID_city', axis=1))
genes_data.shape

(331, 633)

In [5]:
# categorización de la respuesta
le = LabelEncoder()
le.fit(otus_y)
#le.classes_
y_cat = le.transform(otus_y)

In [6]:
from camda_functions import get_tfidf, get_factorization

tfidf = get_tfidf(genes_data)

In [7]:
fact_obj = get_factorization(data=tfidf, n_comp=100, nmf=True)

In [8]:
lsa = fact_obj.transform(tfidf)
lsa.shape

(331, 100)

In [9]:
lsa

array([[0.02158067, 0.09271663, 0.        , ..., 0.10572152, 0.        ,
        0.        ],
       [0.04652969, 0.12062452, 0.        , ..., 0.05317137, 0.        ,
        0.        ],
       [0.02723729, 0.10108946, 0.        , ..., 0.02860653, 0.        ,
        0.00061305],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.09412978, 0.03998854, 0.        , ..., 0.03187948, 0.        ,
        0.05036254],
       [0.        , 0.02973627, 0.        , ..., 0.04179061, 0.01286688,
        0.01838548]])

In [ ]:
# guardar modelo
from camda_functions import save_pickle_model
pkl_file = 'nmf2.pkl'
f_path = os.path.join(ruta_actual, carpeta_data)
save_pickle_model(fact_obj, f_path, pkl_file)

In [17]:
from umap import UMAP
import plotly.express as px

reducer = UMAP(n_neighbors=25, n_components=2)
embedding = reducer.fit_transform(lsa)

umap_proj = pd.DataFrame({'x1': embedding[:,0], 'x2': embedding[:,1], 'ID_city': otus_y})

fig = px.scatter(umap_proj, x='x1', y='x2', hover_data=['ID_city'], color = 'ID_city', title='UMAP plot')
fig.update_layout(autosize=False, width=700, height=700)
fig.show()

In [17]:
latent_names = [f"t {i}" for i in range(lsa.shape[1])]
encoding_matrix = pd.DataFrame(fact_obj.components_, index=latent_names, columns=genes_data.columns).T
encoding_matrix

,t 0,t 1,t 2,t 3,t 4,t 5,t 6,t 7,t 8,t 9,...,t 140,t 141,t 142,t 143,t 144,t 145,t 146,t 147,t 148,t 149
ARO,,,,,,,,,,,,,,,,,,,,,
ARO:3000535,0.558493,-0.071489,-0.328334,0.200104,0.071907,0.230837,-0.314839,-0.194301,-0.082825,-0.124104,...,-0.009112,-0.003171,0.005240,-0.009344,0.016431,-0.001684,0.011953,0.007938,-0.005054,0.006383
ARO:3005091,0.254556,0.116761,-0.046097,0.134876,-0.103598,0.134588,0.130892,-0.073765,0.438827,0.185178,...,-0.053011,-0.002865,0.022296,-0.023421,-0.026325,-0.025642,-0.029546,-0.008811,0.007689,0.020551
ARO:3002987,0.186554,-0.000342,-0.084842,0.255311,0.000325,0.111999,0.397939,0.159532,0.099871,-0.006797,...,0.015797,0.001859,-0.025050,-0.010722,-0.012534,0.036808,0.005326,-0.009494,-0.062720,-0.028818
ARO:3003980,0.322020,-0.145948,-0.155885,-0.343195,0.101113,0.218001,0.187641,0.109489,-0.280264,0.187159,...,0.002076,-0.021707,0.007830,-0.020172,-0.004743,-0.003203,-0.025112,0.012116,0.026382,0.031732
ARO:3003665,0.126629,0.306169,0.065108,-0.025039,-0.103387,0.062208,-0.096734,0.342162,0.149370,-0.064097,...,0.035649,0.014475,0.022674,-0.019712,0.034566,-0.009690,0.027408,0.028830,-0.025922,-0.041149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ARO:3006830,0.000088,-0.000375,-0.000097,-0.000190,0.000389,-0.000338,0.000330,0.000289,0.000437,-0.000374,...,-0.004871,-0.022931,-0.024793,-0.000335,-0.003265,0.004769,0.019876,0.019101,0.003923,-0.000247
ARO:3001767,0.000043,0.000231,-0.000087,-0.000492,0.000082,0.000859,0.000187,-0.000245,0.000149,0.000320,...,0.000255,0.000260,-0.000090,0.002129,0.003471,0.000625,-0.003144,-0.004301,0.001735,0.003102
ARO:3006173,0.000029,-0.000093,-0.000122,-0.000025,0.000098,-0.000209,0.000034,0.000156,-0.000546,0.000371,...,-0.000808,0.003054,-0.000441,-0.002032,0.002875,-0.000307,0.001719,-0.002025,-0.000161,0.001289


In [18]:
from camda_functions import save_pickle_model, load_from_pickle

In [20]:
# guardar modelo

pkl_file = 'svd.pkl'
f_path = os.path.join(ruta_actual, carpeta_data)
save_pickle_model(fact_obj, f_path, pkl_file)

In [22]:
# cargar modelo

ff = load_from_pickle(f_path, pkl_file)

aa = ff.transform(tfidf)
aa.shape

(365, 150)

In [112]:
carpeta_data = "data"
nombre_archivo = "relative_order_assembly.csv"

# Obtén la ruta absoluta de la carpeta actual
ruta_actual = os.getcwd()

# Construye la ruta completa al archivo CSV dentro de la carpeta "data"
ruta_archivo = os.path.join(ruta_actual, carpeta_data, nombre_archivo)

otus = pd.read_csv(ruta_archivo, index_col="ID")
otus_y = otus["ID_city"]
lsa_df = pd.DataFrame(lsa, index=genes_data.index, columns=latent_names)
lsa_city = lsa_df.join(otus_y)
lsa_city

,t 0,t 1,t 2,t 3,t 4,t 5,t 6,t 7,t 8,t 9,...,t 141,t 142,t 143,t 144,t 145,t 146,t 147,t 148,t 149,ID_city
CAMDA23_MetaSUB_gCSD16_AKL_1,0.038144,0.000000,0.000000,0.000000,0.051671,0.000000,0.263392,0.000000,0.000000,0.000000,...,0.000000,0.002815,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,AKL
CAMDA23_MetaSUB_gCSD16_AKL_10,0.055738,0.115444,0.002763,0.077434,0.000000,0.001638,0.102421,0.003002,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.002278,0.000000,AKL
CAMDA23_MetaSUB_gCSD16_AKL_11,0.099789,0.213445,0.000000,0.000000,0.000000,0.000000,0.000000,0.003930,0.000283,0.004083,...,0.005675,0.033340,0.0,0.000000,0.0,0.000000,0.001187,0.000000,0.000000,AKL
CAMDA23_MetaSUB_gCSD16_AKL_12,0.007929,0.006624,0.002186,0.000000,0.125901,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.287065,0.000000,0.082294,0.000000,AKL
CAMDA23_MetaSUB_gCSD16_AKL_13,0.106544,0.004779,0.000928,0.000400,0.000000,0.000000,0.000000,0.001437,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.008757,0.000000,0.000000,0.000000,AKL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CAMDA23_MetaSUB_gCSD17_ZRH_5,0.036990,0.023722,0.000000,0.000000,0.016006,0.000000,0.016526,0.000000,0.000000,0.011774,...,0.228309,0.000000,0.0,0.028847,0.0,0.000000,0.000000,0.000000,0.000000,ZRH
CAMDA23_MetaSUB_gCSD17_ZRH_6,0.042664,0.000000,0.000228,0.005101,0.020801,0.000000,0.000000,0.000000,0.000000,0.009719,...,0.196662,0.000000,0.0,0.000000,0.0,0.000000,0.097899,0.000000,0.000000,ZRH
CAMDA23_MetaSUB_gCSD17_ZRH_7,0.117979,0.000000,0.001833,0.000000,0.000644,0.000000,0.000000,0.000000,0.000000,0.003039,...,0.132508,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,ZRH
CAMDA23_MetaSUB_gCSD17_ZRH_8,0.000000,0.000000,0.000859,0.000000,0.000000,0.001157,0.000000,0.000000,0.000000,0.012895,...,0.000000,0.000000,0.0,0.082313,0.0,0.000000,0.000000,0.011768,0.000000,ZRH
